In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib

In [2]:
# load good stuff
%matplotlib inline
%load_ext oct2py.ipython

In [3]:
# Add niak hcp and psomm to oactve path
%octave addpath(genpath('~/git/Misc'));
%octave build_path hcp niak psom

Adding library hcp to the search path.

Adding library niak to the search path.

Adding library psom to the search path.

In [4]:
# Set path
path_root = '/home/yassinebha/Drive/QC_zooniverse/'
#path_root = '/media/yassinebha/database29/Drive/QC_zooniverse/'
path_outline = os.path.join(path_root ,'template_layout/brain_outline_bold/')
path_layout = os.path.join(path_root ,'template_layout/layout/')
%octave_push  path_root path_outline path_layout

In [5]:
%%octave
# invert mask func
in.source = [path_root 'template_layout/test_data/HC0040013/func_HC0040013_mean_stereonl.nii.gz'];
out = [path_root 'template_layout/test_data/HC0040013/HC0040013_test_inv_func.nii.gz'];
opt.only_mask = false;
[in,out,opt] = zoo_brick_invert_contrast(in,out,opt);

In [6]:
%%octave
# invert mask anat
in.source = [path_root 'template_layout/test_data/HC0040013/anat_HC0040013_nuc_stereonl.nii.gz'];
out = [path_root 'template_layout/test_data/HC0040013/HC0040013_test_inv_anat.nii.gz'];
opt.only_mask = true;
[in,out,opt] = zoo_brick_invert_contrast(in,out,opt);

In [7]:
%octave [path_layout 'layout_anat.nii.gz']

ans = /home/yassinebha/Drive/QC_zooniverse/template_layout/layout/layout_anat.nii.gz

'/home/yassinebha/Drive/QC_zooniverse/template_layout/layout/layout_anat.nii.gz'

In [8]:
%%octave
# anat outline
clear in out opt
in.layout=[path_layout 'layout_anat.nii.gz'];
out = '/home/yassinebha/Drive/QC_zooniverse/template_layout/test_anat_layout.nii.gz';
[in,out,opt]= zoo_brick_outline(in,out);


Bad argument "/tmp/niak_tmp_381204710_vol.mnc".

Usage: rawtominc [<options>] <output.mnc> [[<size4>] <size3>] <size2> <size1>

       rawtominc [-help]

ncopen: filename "/tmp/niak_tmp_381204710_vol.mnc": Not a netCDF file

miopen: MINC package entry point

...

ncopen: filename "/tmp/niak_tmp_900845551_vol.mnc": Not a netCDF file

miopen: MINC package entry point

(from miopen): Unable to open file '/tmp/niak_tmp_900845551_vol.mnc'

Error: opening MINC file "/tmp/niak_tmp_900845551_vol.mnc".

Oct2PyError: Octave evaluation error:
error: mx_el_or: nonconformant arguments (op1 is 197x233x189, op2 is 0x0)
error: called from:
    zoo_brick_outline at line 106, column 11

In [31]:
%%octave
#func outline
clear in out opt
in.layout=[path_layout 'layout.nii.gz'];
in.mask_func = [path_outline 'func_mask_average_stereonl.mnc.gz'];
opt.modality = 'func';
out = '/home/yassinebha/Drive/QC_zooniverse/template_layout/test_func_layout.nii.gz';

[in,out,opt]= zoo_brick_outline(in,out,opt);

Transforming slices:.............................................................................................................................................................................................Done


In [9]:
%%octave
#local pipeline report test
clear all
% build path
addpath(genpath('../Misc'))
build_path  niak psom zooniverse_qc  

# Set path
path_root = '/home/yassinebha/Drive/QC_zooniverse/';
#path_root = '/media/yassinebha/database29/Drive/QC_zooniverse/';
path_outline = [path_root 'template_layout/brain_outline_bold/'];
path_layout = [path_root 'template_layout/layout/'];

%X_0015002 
in.ind.anat.X_0015002 = [path_root 'template_layout/test_data/X_0015002/anat_X_0015002_nuc_stereonl.nii.gz'];
in.ind.func.X_0015002  = [path_root 'template_layout/test_data/X_0015002/func_X_0015002_mean_stereonl.nii.gz'];

in.group.mask_func_group = [path_root 'template_layout/test_data/X_0015002/func_mask_group_stereonl.nii.gz'];
in.group.avg_mask_func = [path_root 'template_layout/test_data/X_0015002/func_mask_average_stereonl.nii.gz'];
in.template.anat_outline = [path_layout 'layout_anat.nii.gz'];
in.template.func_outline = [path_layout 'layout_func.nii.gz']'';

%opt.invert_contrast.perc_min = 0.0001;
%opt.invert_contrast.perc_max = 0.9999;

opt.invert_contrast.perc_min = 0.1;
opt.invert_contrast.perc_max = 0.9999;

opt.folder_out = [path_root 'template_layout/test_data/zooqc_adhd200_test5'];
pipeline = zoo_report_fmri_preprocess(in,opt);


Adding library niak to the search path.

Adding library psom to the search path.

Adding library zooniverse_qc to the search path.


Logs will be stored in /home/yassinebha/Drive/QC_zooniverse/template_layout/test_data/zooqc_adhd200_test5/logs/
Generating dependencies ...
   Percentage completed :  0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100- 0.03 sec
Setting up the to-do list ...
   I found 7 job(s) to do, and 23 job(s) already completed.
Deamon started on 28-May-2017 08:43:25
28-May-2017 08:43:25 Starting the pipeline manager...
28-May-2017 08:43:25 Starting the garbage collector...
28-May-2017 08:43:25 Starting worker number 1...
28-May-2017 08:43:25 Starting worker number 2...
28-May-2017 08:43:25 Starting worker number 3...
28-May-2017 08:43:26 Starting worker number 4...
28-May-2017 08:43:26 Starting worker number 5...
28-May-2017 08:43:26 Starting worker number 6...
28-May-2017 08:43:26 Starting worker number 7...

Pipeline started on 28-May-2017 08:43:26
user:

In [10]:
%%octave
#local pipeline report test
clear all
% build path
addpath(genpath('../Misc'))
build_path  niak psom zooniverse_qc  

# Set path
path_root = '/home/yassinebha/Drive/QC_zooniverse/';
#path_root = '/media/yassinebha/database29/Drive/QC_zooniverse/';
path_outline = [path_root 'template_layout/brain_outline_bold/'];
path_layout = [path_root 'template_layout/layout/'];

%HC0040013 
in.ind.anat.HC0040013 = [path_root 'template_layout/test_data/HC0040013/anat_HC0040013_nuc_stereonl.nii.gz'];
in.ind.func.HC0040013  = [path_root 'template_layout/test_data/HC0040013/func_HC0040013_mean_stereonl.nii.gz'];
%HC0040020 
in.ind.anat.HC0040020 = [path_root 'template_layout/test_data/HC0040020/anat_HC0040020_nuc_stereonl.nii.gz'];
in.ind.func.HC0040020  = [path_root 'template_layout/test_data/HC0040020/func_HC0040020_mean_stereonl.nii.gz'];

in.group.mask_func_group = [path_root 'template_layout/test_data/HC0040013/func_mask_group_stereonl.nii.gz'];
in.group.avg_mask_func = [path_root 'template_layout/test_data/group_coregistration/func_mask_average_stereonl.nii.gz'];
in.template.anat_outline = [path_layout 'layout_anat.nii.gz'];
in.template.func_outline = [path_layout 'layout_func.nii.gz']'';

opt.invert_contrast.perc_min = 0.0001;
opt.invert_contrast.perc_max = 0.9999;
opt.folder_out = [path_root 'template_layout/test_data/zooqc_adhd200_test4'];
pipeline = zoo_report_fmri_preprocess(in,opt);


Adding library niak to the search path.

Adding library psom to the search path.

Adding library zooniverse_qc to the search path.


Logs will be stored in /home/yassinebha/Drive/QC_zooniverse/template_layout/test_data/zooqc_adhd200_test4/logs/
Generating dependencies ...
   Percentage completed :  0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100- 0.04 sec
Setting up the to-do list ...
   I found 45 job(s) to do.
I could not find any log file. This pipeline has not been started (yet?). Press CTRL-C to cancel.
Deamon started on 28-May-2017 10:22:45
28-May-2017 10:22:45 Starting the pipeline manager...
28-May-2017 10:22:45 Starting the garbage collector...
28-May-2017 10:22:45 Starting worker number 1...
28-May-2017 10:22:45 Starting worker number 2...
28-May-2017 10:22:45 Starting worker number 3...
28-May-2017 10:22:45 Starting worker number 4...
28-May-2017 10:22:45 Starting worker number 5...
28-May-2017 10:22:45 Starting worker number 6...

Pipeline started on 28-May-20

In [2]:
%%octave
psom_pipeline_visu('/home/yassinebha/Drive/QC_zooniverse/template_layout/test_data/zooqc_adhd200_test4/logs/','log','invert_anat_mask');



***************************************************
  Log file of job invert_anat_mask (status failed) 
***************************************************


******************************
Log of the (octave) job : invert_anat_mask
Started on 28-May-2017 08:18:12
User: yassinebha
host : yassinebha-Lemur-Ultra
system : unix
******************************


Something went bad ... the job has FAILED !
The last error message occured was :
Couldn't find any file fitting the description 
File /home/yassinebha/git/niak/commands/read_write/niak_read_vol.m at line 188
File /home/yassinebha/git/psom/psom_run_job.m at line 185
File /home/yassinebha/git/psom/psom_run_job.m at line 106
File /home/yassinebha/git/psom/psom_worker.m at line 161


The output file or directory /home/yassinebha/Drive/QC_zooniverse/template_layout/test_data/zooqc_adhd200_test4/group/anat_group_mask_sym.nii.gz has not been generated!

**********************************************
28-May-2017 08:18:12 : The job has FAILE

In [ ]:
#run on guillimin cobree
clear all
addpath(genpath('/home/yassinebha/git/Misc'))
build_path zooniverse_qc

path_fmri_preproc ='/home/yassinebha/database/cobre_fmri_preprocess_nii_20160920/';
in = zoo_grab_report_preprocess(path_fmri_preproc);
in.template.anat_outline = '/home/yassinebha/layout_anat.nii.gz';
in.template.func_outline = '/home/yassinebha/layout_func.nii.gz';

opt.invert_contrast.perc_min = 0.0001;
opt.invert_contrast.perc_max = 0.9999;
opt.folder_out =['/gs/project/gsf-624-aa/projects/zooniverse_qc/zooqc_report_cobre_' date];
pipeline = zoo_report_fmri_preprocess(in,opt);

In [ ]:
#run on guillimin adhd
clear all
addpath(genpath('/home/yassinebha/git/Misc'))
build_path zooniverse_qc
path_fmri_preproc ='/gs/project/gsf-624-aa/preprocess/adhd200/niak/fmri_preprocess_old/';
files_in  = niak_grab_qc_fmri_preprocess(path_fmri_preproc);
in.ind.anat = files_in.anat;
in.ind.func = files_in.func;
in.group.mask_func_group = ['/gs/project/gsf-624-aa/preprocess/adhd200/niak/fmri_preprocess/mask/func_mask_group_stereonl.nii.gz'];
in.group.avg_mask_func = ['/gs/project/gsf-624-aa/preprocess/adhd200/niak/fmri_preprocess/mask/func_mask_average_stereonl.nii.gz'];
in.template.anat_outline = '/home/yassinebha/layout_anat.nii.gz';
in.template.func_outline = '/home/yassinebha/layout_func.nii.gz';

opt.invert_contrast.perc_min = 0.1;
opt.invert_contrast.perc_max = 0.9999;
opt.folder_out =['/gs/project/gsf-624-aa/projects/zooniverse_qc/zooqc_report_adhd_' date];

pipeline = zoo_report_fmri_preprocess(in,opt);

In [ ]:
# search best coordinate or anat an func montage 
#coord_old =[-30 , -65 , -6 ;
#             -8 , -25 ,  10 ;
#             30 ,  45 ,  60];          
             
coord_anat =[-50 , -65 , -6 ;
             -8 , -20 ,  13 ;
             30 ,  54 ,  58];
             
coord_func =[-50 , -57 , 5 ;
             -8 , -20 ,  19 ;
             30 ,  45 ,  58];